In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import sklearn as skl
import tensorflow as tf
import yfinance as yf
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
from pathlib import Path
from sklearn.linear_model import LinearRegression
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import json
import os
import pymongo
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
import seaborn as sb
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [23]:
Soybeans=yf.Ticker("ZS=F")
Corn=yf.Ticker("ZC=F")
Wheat=yf.Ticker("ZW=F")
Cotton=yf.Ticker("CT=F")
Crude=yf.Ticker("CL=F")
USDBRL=yf.Ticker("BRL=X")
EURUSD=yf.Ticker("EURUSD=X")
IR=yf.Ticker("^TNX")
Sugar=yf.Ticker("SB=F")
SoyOil=yf.Ticker("ZL=F")

Hold_Signal=-.05
window = 90  # trading days in rolling window
dpy = 252  # trading days per year
ann_factor = dpy / window

In [24]:
Sugar_H=Sugar.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Sugar_H["Range"]=Sugar_H["High"]-Sugar_H["Low"]
Sugar_H=Sugar_H.drop(["High","Low"], axis=1)

Sugar_H['log_rtn'] = np.log(Sugar_H['Close']).diff()
Sugar_H['real_var'] = np.square(Sugar_H['log_rtn']).rolling(window).sum() * ann_factor
Sugar_H['Vol'] = np.sqrt(Sugar_H['real_var'])

Sugar_P=Sugar_H[:len(Sugar_H)-window]
Sugar_P=Sugar_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Sugar_H=Sugar_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Sugar_P=Sugar_P.drop("Date",axis=1)

Sugar_H["Price_Change"]=(Sugar_H["Close"]-Sugar_P["Close"])/Sugar_P["Close"]
Sugar_H["Buy/Hold"]=np.where(Sugar_H["Price_Change"]< Hold_Signal, "Hold","Buy")

Sugar_P

,Close,Range,Vol
0,14.72,0.570000,0.291503
1,14.97,0.810000,0.292519
2,14.36,0.500000,0.284243
3,14.40,0.500000,0.284261
4,14.89,0.330000,0.279296
...,...,...,...
5061,9.20,0.160001,0.614862
5062,9.22,0.230000,0.614932
5063,9.17,0.320000,0.614873
5064,8.83,0.230000,0.612354


In [25]:
SoyOil_H=SoyOil.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
SoyOil_H["Range"]=SoyOil_H["High"]-SoyOil_H["Low"]
SoyOil_H=SoyOil_H.drop(["High","Low"], axis=1)

SoyOil_H['log_rtn'] = np.log(SoyOil_H['Close']).diff()


SoyOil_H['real_var'] = np.square(SoyOil_H['log_rtn']).rolling(window).sum() * ann_factor
SoyOil_H['Vol'] = np.sqrt(SoyOil_H['real_var'])


SoyOil_P=SoyOil_H[:len(SoyOil_H)-window]
SoyOil_P=SoyOil_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
SoyOil_H=SoyOil_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
SoyOil_P=SoyOil_P.drop("Date",axis=1)

SoyOil_H["Price_Change"]=(SoyOil_H["Close"]-SoyOil_P["Close"])/SoyOil_P["Close"]
SoyOil_H["Buy/Hold"]=np.where(SoyOil_H["Price_Change"]< Hold_Signal, "Hold","Buy")

SoyOil_P

,Close,Range,Vol
0,33.779999,0.790001,0.230207
1,33.189999,0.590000,0.229255
2,33.610001,0.780003,0.228385
3,33.060001,0.700001,0.228844
4,33.419998,0.910000,0.228374
...,...,...,...
4964,15.540000,0.340000,0.208599
4965,15.580000,0.230000,0.209357
4966,15.520000,0.150000,0.212498
4967,15.560000,0.179999,0.229789


In [26]:
Soybeans_H=Soybeans.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Soybeans_H["Range"]=Soybeans_H["High"]-Soybeans_H["Low"]
Soybeans_H=Soybeans_H.drop(["High","Low"], axis=1)

Soybeans_H['log_rtn'] = np.log(Soybeans_H['Close']).diff()

# Var Swap (returns are not demeaned)
Soybeans_H['real_var'] = np.square(Soybeans_H['log_rtn']).rolling(window).sum() * ann_factor
Soybeans_H['Vol'] = np.sqrt(Soybeans_H['real_var'])


Soybeans_P=Soybeans_H[:len(Soybeans_H)-window]
Soybeans_P=Soybeans_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Soybeans_H=Soybeans_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Soybeans_P=Soybeans_P.drop("Date",axis=1)

Soybeans_H["Price_Change"]=(Soybeans_H["Close"]-Soybeans_P["Close"])/Soybeans_P["Close"]
Soybeans_H["Buy/Hold"]=np.where(Soybeans_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Soybeans_H["Past_Price"]=Soybeans_P["Close"]
#Soybeans_H=Soybeans_H.set_index('Date')
#Soybeans_P=Soybeans_P.set_index('Date')
#Soybeans_H["Buy/Hold"].hist(figsize = (12,10))
#plt.show()
Soybeans_P

,Close,Range,Vol
0,1059.00,17.50,0.172572
1,1050.75,11.50,0.172270
2,1056.50,15.25,0.172052
3,1051.75,11.25,0.172080
4,1057.25,25.00,0.171875
...,...,...,...
4940,459.75,1.25,0.169929
4941,461.25,1.50,0.170212
4942,459.75,2.25,0.170160
4943,467.00,2.00,0.168136


In [27]:
Corn_H=Corn.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Corn_H["Range"]=Corn_H["High"]-Corn_H["Low"]
Corn_H=Corn_H.drop(["High","Low"], axis=1)


Corn_H['log_rtn'] = np.log(Corn_H['Close']).diff()

# Var Swap (returns are not demeaned)
Corn_H['real_var'] = np.square(Corn_H['log_rtn']).rolling(window).sum() * ann_factor
Corn_H['Vol'] = np.sqrt(Corn_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Corn_P=Corn_H[:len(Corn_H)-90]
Corn_P=Corn_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Corn_H=Corn_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Corn_P=Corn_P.drop("Date",axis=1)

Corn_H["Price_Change"]=(Corn_H["Close"]-Corn_P["Close"])/Corn_P["Close"]
Corn_H["Buy/Hold"]=np.where(Corn_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Corn_H["Past_Price"]=Corn_P["Close"]
#Corn_H=Corn_H.set_index('Date')
#Corn_P=Corn_P.set_index('Date')
#Corn_H["Buy/Hold"].hist(figsize = (12,10))
#plt.show()
Corn_P

,Close,Range,Vol
0,401.0,8.00,0.234556
1,397.5,4.75,0.234100
2,398.5,5.75,0.236894
3,398.5,11.25,0.236926
4,401.5,14.75,0.237179
...,...,...,...
4948,220.0,1.00,0.183245
4949,218.0,0.00,0.183278
4950,215.5,0.00,0.185854
4951,210.0,0.00,0.181116


In [28]:
Wheat_H=Wheat.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Wheat_H["Range"]=Wheat_H["High"]-Wheat_H["Low"]
Wheat_H=Wheat_H.drop(["High","Low"], axis=1)


Wheat_H['log_rtn'] = np.log(Wheat_H['Close']).diff()

# Var Swap (returns are not demeaned)
Wheat_H['real_var'] = np.square(Wheat_H['log_rtn']).rolling(window).sum() * ann_factor
Wheat_H['Vol'] = np.sqrt(Wheat_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Wheat_P=Wheat_H[:len(Wheat_H)-90]
Wheat_P=Wheat_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Wheat_H=Wheat_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Wheat_P=Wheat_P.drop("Date",axis=1)

Wheat_H["Price_Change"]=(Wheat_H["Close"]-Wheat_P["Close"])/Wheat_P["Close"]
Wheat_H["Buy/Hold"]=np.where(Wheat_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Wheat_H["Past_Price"]=Wheat_P["Close"]
#Wheat_H=Wheat_H.set_index('Date')
#Wheat_P=Wheat_P.set_index('Date')

Wheat_P

,Close,Range,Vol
0,608.00,12.50,0.279679
1,607.50,18.50,0.283180
2,598.50,12.00,0.282717
3,603.75,12.75,0.282817
4,608.75,13.75,0.282501
...,...,...,...
4936,252.50,2.00,0.245302
4937,256.25,1.75,0.244064
4938,256.50,1.75,0.244438
4939,261.00,1.00,0.244068


In [29]:
Cotton_H=Cotton.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Cotton_H["Range"]=Cotton_H["High"]-Cotton_H["Low"]
Cotton_H=Cotton_H.drop(["High","Low"], axis=1)


Cotton_H['log_rtn'] = np.log(Cotton_H['Close']).diff()

# Var Swap (returns are not demeaned)
Cotton_H['real_var'] = np.square(Cotton_H['log_rtn']).rolling(window).sum() * ann_factor
Cotton_H['Vol'] = np.sqrt(Cotton_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Cotton_P=Cotton_H[:len(Cotton_H)-90]
Cotton_P=Cotton_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Cotton_H=Cotton_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Cotton_P=Cotton_P.drop("Date",axis=1)


Cotton_H["Price_Change"]=(Cotton_H["Close"]-Cotton_P["Close"])/Cotton_P["Close"]
Cotton_H["Buy/Hold"]=np.where(Cotton_H["Price_Change"]< Hold_Signal, "Hold","Buy")
#Cotton_H["Past_Price"]=Cotton_P["Close"]
#Cotton_H=Cotton_H.set_index('Date')
#Cotton_P=Cotton_P.set_index('Date')
Cotton_P

,Close,Range,Vol
0,70.239998,1.599998,0.199291
1,68.730003,1.029999,0.198953
2,68.919998,1.680000,0.198941
3,69.820000,0.879997,0.201191
4,70.169998,2.549995,0.201494
...,...,...,...
5100,62.070000,1.299999,0.257533
5101,62.619999,0.550003,0.263868
5102,62.750000,1.489998,0.264379
5103,62.230000,1.000000,0.265405


In [30]:
Crude_H=Crude.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
Crude_H["Range"]=Crude_H["High"]-Crude_H["Low"]
Crude_H=Crude_H.drop(["High","Low"], axis=1)


Crude_H['log_rtn'] = np.log(Crude_H['Close']).diff()

# Var Swap (returns are not demeaned)
Crude_H['real_var'] = np.square(Crude_H['log_rtn']).rolling(window).sum() * ann_factor
Crude_H['Vol'] = np.sqrt(Crude_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

Crude_P=Crude_H[:len(Crude_H)-90]
Crude_P=Crude_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Crude_H=Crude_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
Crude_P=Crude_P.drop("Date",axis=1)
Crude_P

,Close,Range,Vol
0,37.660000,1.750000,0.367426
1,36.810001,3.510002,0.365571
2,35.790001,1.389999,0.363855
3,36.169998,2.840000,0.377144
4,37.389999,2.039997,0.373056
...,...,...,...
4856,27.719999,0.850000,0.480530
4857,27.350000,1.250000,0.480065
4858,28.000000,1.000000,0.480317
4859,28.200001,0.930000,0.480676


In [31]:
USDBRL_H=USDBRL.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)

USDBRL_H["Range"]=USDBRL_H["High"]-USDBRL_H["Low"]
USDBRL_H=USDBRL_H.drop(["High","Low"], axis=1)


USDBRL_H['log_rtn'] = np.log(USDBRL_H['Close']).diff()

# Var Swap (returns are not demeaned)
USDBRL_H['real_var'] = np.square(USDBRL_H['log_rtn']).rolling(window).sum() * ann_factor
USDBRL_H['Vol'] = np.sqrt(USDBRL_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

USDBRL_P=USDBRL_H[:len(USDBRL_H)-90]
USDBRL_P=USDBRL_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
USDBRL_H=USDBRL_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()#[:len(Soybeans_H)]
USDBRL_P=USDBRL_P.drop("Date",axis=1)
USDBRL_P

,Close,Range,Vol
0,5.3900,0.077600,0.188734
1,5.4139,0.071600,0.188908
2,5.3847,0.069791,0.188834
3,5.3627,0.154900,0.189189
4,5.5234,0.119000,0.183230
...,...,...,...
3862,2.8840,0.000000,0.079259
3863,2.8840,0.014000,0.079424
3864,2.8700,0.016000,0.079301
3865,2.8740,0.005000,0.079267


In [32]:
EURUSD_H=EURUSD.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
EURUSD_H["Range"]=EURUSD_H["High"]-EURUSD_H["Low"]
EURUSD_H=EURUSD_H.drop(["High","Low"], axis=1)


EURUSD_H['log_rtn'] = np.log(EURUSD_H['Close']).diff()

# Var Swap (returns are not demeaned)
EURUSD_H['real_var'] = np.square(EURUSD_H['log_rtn']).rolling(window).sum() * ann_factor
EURUSD_H['Vol'] = np.sqrt(EURUSD_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

EURUSD_P=EURUSD_H[:len(EURUSD_H)-90]
EURUSD_P=EURUSD_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
EURUSD_H=EURUSD_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()#[:len(Soybeans_H)]
EURUSD_P=EURUSD_P.drop("Date",axis=1)

EURUSD_P

,Close,Range,Vol
0,1.178030,0.006341,0.067620
1,1.181879,0.008563,0.068060
2,1.182732,0.006279,0.068278
3,1.189301,0.012177,0.068343
4,1.181754,0.009440,0.067522
...,...,...,...
4280,1.208605,0.004604,0.123308
4281,1.207700,0.015008,0.124165
4282,1.216604,0.013900,0.123692
4283,1.213106,0.015698,0.123687


In [33]:
IR_H=IR.history(period="Max").drop(["Dividends", "Stock Splits","Open","Volume"], axis=1)
IR_H["Range"]=IR_H["High"]-IR_H["Low"]
IR_H=IR_H.drop(["High","Low"], axis=1)


IR_H['log_rtn'] = np.log(IR_H['Close']).diff()

# Var Swap (returns are not demeaned)
IR_H['real_var'] = np.square(IR_H['log_rtn']).rolling(window).sum() * ann_factor
IR_H['Vol'] = np.sqrt(IR_H['real_var'])

# Classical (returns are demeaned, dof=1)
#df['real_var'] = df['log_rtn'].rolling(window).var() * ann_factor
#df['real_vol'] = np.sqrt(df['real_var'])

IR_P=IR_H[:len(IR_H)-90]
IR_P=IR_P.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()
IR_H=IR_H.sort_values(by="Date", ascending=False).dropna().drop(["log_rtn","real_var"],axis=1).reset_index()#[:len(Soybeans_H)]
IR_P=IR_P.drop("Date",axis=1)
IR_P

,Close,Range,Vol
0,0.882,0.025,0.656190
1,0.849,0.027,0.660260
2,0.860,0.049,0.660369
3,0.835,0.073,0.661255
4,0.781,0.037,0.651831
...,...,...,...
14593,3.870,0.000,0.082033
14594,3.860,0.000,0.082024
14595,3.870,0.000,0.082863
14596,3.850,0.000,0.084082


In [34]:
Sugar_H=Sugar_H.rename(columns={"Range": "Sugar Range", "Close": "Sugar Close", "Vol" : "Sugar Vol"})[:len(USDBRL_H)]
SoyOil_H=SoyOil_H.rename(columns={"Range": "SoyOil Range", "Close": "SoyOil Close", "Vol" : "SoyOil Vol"})[:len(USDBRL_H)]
Sugar_P=Sugar_P.rename(columns={"Range": "Sugar P_Range", "Close": "Sugar P_Close", "Vol" : "Sugar P_Vol"})[:len(USDBRL_P)]
SoyOil_P=SoyOil_P.rename(columns={"Range": "SoyOil P_Range", "Close": "SoyOil P_Close", "Vol" : "SoyOil P_Vol"})[:len(USDBRL_P)]

In [35]:
Soybeans_H=Soybeans_H.rename(columns={"Range": "Soybeans Range", "Close": "Soybeans Close", "Vol" : "Soybeans Vol"})[:len(USDBRL_H)]
Corn_H=Corn_H.rename(columns={"Range": "Corn Range", "Close": "Corn Close", "Vol" : "Corn Vol"})[:len(USDBRL_H)]

Wheat_H=Wheat_H.rename(columns={"Range": "Wheat Range", "Close": "Wheat Close", "Vol" : "Wheat Vol"})[:len(USDBRL_H)]
Cotton_H=Cotton_H.rename(columns={"Range": "Cotton Range","Close": "Cotton Close", "Vol" : "Cotton Vol"})[:len(USDBRL_H)]
Crude_H=Crude_H.rename(columns={"Range": "Crude Range","Close": "Crude Close", "Vol" : "Crude Vol"})[:len(USDBRL_H)]
USDBRL_H=USDBRL_H.rename(columns={"Range": "USDBRL Range", "Close": "USDBRL Close", "Vol" : "USDBRL Vol"})[:len(USDBRL_H)]
EURUSD_H=EURUSD_H.rename(columns={"Range": "EURUSD Range", "Close": "EURUSD Close", "Vol" : "EURUSD Vol"})[:len(USDBRL_H)]
IR_H=IR_H.rename(columns={"Range": "IR Range", "Close": "IR Close", "Vol" : "IR Vol"})[:len(USDBRL_H)]

In [36]:
Soybeans_P=Soybeans_P.rename(columns={"Range": "Soybeans P_Range", "Close": "Soybeans P_Close", "Vol" : "Soybeans P_Vol"})[:len(USDBRL_P)]
Corn_P=Corn_P.rename(columns={"Range": "Corn P_Range", "Close": "Corn P_Close", "Vol" : "Corn P_Vol"})[:len(USDBRL_P)]

Wheat_P=Wheat_P.rename(columns={"Range": "Wheat P_Range", "Close": "Wheat P_Close", "Vol" : "Wheat P_Vol"})[:len(USDBRL_P)]
Cotton_P=Cotton_P.rename(columns={"Range": "Cotton P_Range","Close": "Cotton P_Close", "Vol" : "Cotton P_Vol"})[:len(USDBRL_P)]
Crude_P=Crude_P.rename(columns={"Range": "Crude P_Range","Close": "Crude P_Close", "Vol" : "Crude P_Vol"})[:len(USDBRL_P)]
USDBRL_P=USDBRL_P.rename(columns={"Range": "USDBRL P_Range", "Close": "USDBRL P_Close", "Vol" : "USDBRL P_Vol"})[:len(USDBRL_P)]
EURUSD_P=EURUSD_P.rename(columns={"Range": "EURUSD P_Range", "Close": "EURUSD P_Close", "Vol" : "EURUSD P_Vol"})[:len(USDBRL_P)]
IR_P=IR_P.rename(columns={"Range": "IR P_Range", "Close": "IR P_Close", "Vol" : "IR P_Vol"})[:len(USDBRL_P)]

In [37]:
Prices_S = [Corn_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P, SoyOil_P, Sugar_P]
Prices_C= [Soybeans_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P, SoyOil_P, Sugar_P]
Prices_W= [Soybeans_P, Corn_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P,SoyOil_P, Sugar_P]
Prices_Ctn= [Soybeans_P, Wheat_P, Corn_P, Crude_P, USDBRL_P, EURUSD_P, IR_P, SoyOil_P, Sugar_P]
Prices_Sb=[Corn_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P,Soybeans_P,SoyOil_P]
Prices_SO=[Corn_P, Wheat_P, Cotton_P, Crude_P, USDBRL_P, EURUSD_P, IR_P,Soybeans_P,Sugar_P]

In [38]:
for price in Prices_S:
    Soybeans_H=Soybeans_H.merge(price, how="outer", left_index=True, right_index=True)
    
Soybeans_H=Soybeans_H.drop("Date", axis=1)
Soybeans_H


,Soybeans Close,Soybeans Range,Soybeans Vol,Price_Change,Buy/Hold,Corn P_Close,Corn P_Range,Corn P_Vol,Wheat P_Close,Wheat P_Range,...,EURUSD P_Vol,IR P_Close,IR P_Range,IR P_Vol,SoyOil P_Close,SoyOil P_Range,SoyOil P_Vol,Sugar P_Close,Sugar P_Range,Sugar P_Vol
0,1412.75,15.50,0.207640,0.334042,Buy,401.0,8.00,0.234556,608.00,12.50,...,0.067620,0.882,0.025,0.656190,33.779999,0.790001,0.230207,14.72,0.57,0.291503
1,1417.75,17.25,0.207968,0.349274,Buy,397.5,4.75,0.234100,607.50,18.50,...,0.068060,0.849,0.027,0.660260,33.189999,0.590000,0.229255,14.97,0.81,0.292519
2,1423.25,12.25,0.208067,0.347137,Buy,398.5,5.75,0.236894,598.50,12.00,...,0.068278,0.860,0.049,0.660369,33.610001,0.780003,0.228385,14.36,0.50,0.284243
3,1419.50,21.75,0.208157,0.349655,Buy,398.5,11.25,0.236926,603.75,12.75,...,0.068343,0.835,0.073,0.661255,33.060001,0.700001,0.228844,14.40,0.50,0.284261
4,1415.75,29.25,0.208293,0.339087,Buy,401.5,14.75,0.237179,608.75,13.75,...,0.067522,0.781,0.037,0.651831,33.419998,0.910000,0.228374,14.89,0.33,0.279296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,629.25,6.50,0.274716,0.123661,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,633.25,7.00,0.275632,0.147712,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,620.75,7.25,0.275312,0.145824,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,623.50,6.00,0.276267,0.167603,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
for price in Prices_C:
    Corn_H=Corn_H.merge(price, how="outer", left_index=True, right_index=True)
    
Corn_H=Corn_H.drop("Date", axis=1)
Corn_H

,Corn Close,Corn Range,Corn Vol,Price_Change,Buy/Hold,Soybeans P_Close,Soybeans P_Range,Soybeans P_Vol,Wheat P_Close,Wheat P_Range,...,EURUSD P_Vol,IR P_Close,IR P_Range,IR P_Vol,SoyOil P_Close,SoyOil P_Range,SoyOil P_Vol,Sugar P_Close,Sugar P_Range,Sugar P_Vol
0,551.50,9.50,0.249532,0.375312,Buy,1059.00,17.50,0.172572,608.00,12.50,...,0.067620,0.882,0.025,0.656190,33.779999,0.790001,0.230207,14.72,0.57,0.291503
1,558.00,6.50,0.249192,0.403774,Buy,1050.75,11.50,0.172270,607.50,18.50,...,0.068060,0.849,0.027,0.660260,33.189999,0.590000,0.229255,14.97,0.81,0.292519
2,554.25,8.75,0.248972,0.390841,Buy,1056.50,15.25,0.172052,598.50,12.00,...,0.068278,0.860,0.049,0.660369,33.610001,0.780003,0.228385,14.36,0.50,0.284243
3,549.50,18.00,0.248555,0.378921,Buy,1051.75,11.25,0.172080,603.75,12.75,...,0.068343,0.835,0.073,0.661255,33.060001,0.700001,0.228844,14.40,0.50,0.284261
4,549.25,7.00,0.248871,0.367995,Buy,1057.25,25.00,0.171875,608.75,13.75,...,0.067522,0.781,0.037,0.651831,33.419998,0.910000,0.228374,14.89,0.33,0.279296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,207.50,1.75,0.235973,-0.005988,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,207.00,1.75,0.243240,0.027295,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,195.25,0.00,0.222758,-0.028607,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,196.25,0.00,0.222681,-0.027261,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
for price in Prices_W:
    Wheat_H=Wheat_H.merge(price, how="outer", left_index=True, right_index=True)
    
Wheat_H=Wheat_H.drop("Date", axis=1)
Wheat_H

,Wheat Close,Wheat Range,Wheat Vol,Price_Change,Buy/Hold,Soybeans P_Close,Soybeans P_Range,Soybeans P_Vol,Corn P_Close,Corn P_Range,...,EURUSD P_Vol,IR P_Close,IR P_Range,IR P_Vol,SoyOil P_Close,SoyOil P_Range,SoyOil P_Vol,Sugar P_Close,Sugar P_Range,Sugar P_Vol
0,634.00,8.00,0.274641,0.042763,Buy,1059.00,17.50,0.172572,401.0,8.00,...,0.067620,0.882,0.025,0.656190,33.779999,0.790001,0.230207,14.72,0.57,0.291503
1,640.00,10.25,0.274192,0.053498,Buy,1050.75,11.50,0.172270,397.5,4.75,...,0.068060,0.849,0.027,0.660260,33.189999,0.590000,0.229255,14.97,0.81,0.292519
2,647.00,16.25,0.274725,0.081036,Buy,1056.50,15.25,0.172052,398.5,5.75,...,0.068278,0.860,0.049,0.660369,33.610001,0.780003,0.228385,14.36,0.50,0.284243
3,645.00,11.75,0.275064,0.068323,Buy,1051.75,11.25,0.172080,398.5,11.25,...,0.068343,0.835,0.073,0.661255,33.060001,0.700001,0.228844,14.40,0.50,0.284261
4,631.75,0.00,0.273211,0.037782,Buy,1057.25,25.00,0.171875,401.5,14.75,...,0.067522,0.781,0.037,0.651831,33.419998,0.910000,0.228374,14.89,0.33,0.279296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,309.50,2.50,0.282267,0.005686,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,315.00,1.75,0.280933,0.016949,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,310.00,2.50,0.279772,0.005677,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,306.25,2.25,0.282926,0.021685,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
for price in Prices_Ctn:
    Cotton_H=Cotton_H.merge(price, how="outer", left_index=True, right_index=True)

Cotton_H=Cotton_H.drop("Date", axis=1)    
Cotton_H

,Cotton Close,Cotton Range,Cotton Vol,Price_Change,Buy/Hold,Soybeans P_Close,Soybeans P_Range,Soybeans P_Vol,Wheat P_Close,Wheat P_Range,...,EURUSD P_Vol,IR P_Close,IR P_Range,IR P_Vol,SoyOil P_Close,SoyOil P_Range,SoyOil P_Vol,Sugar P_Close,Sugar P_Range,Sugar P_Vol
0,86.820000,2.029999,0.250994,0.236048,Buy,1059.00,17.50,0.172572,608.00,12.50,...,0.067620,0.882,0.025,0.656190,33.779999,0.790001,0.230207,14.72,0.57,0.291503
1,86.510002,1.320000,0.253544,0.258693,Buy,1050.75,11.50,0.172270,607.50,18.50,...,0.068060,0.849,0.027,0.660260,33.189999,0.590000,0.229255,14.97,0.81,0.292519
2,86.919998,1.409996,0.253463,0.261172,Buy,1056.50,15.25,0.172052,598.50,12.00,...,0.068278,0.860,0.049,0.660369,33.610001,0.780003,0.228385,14.36,0.50,0.284243
3,86.720001,2.290001,0.254362,0.242051,Buy,1051.75,11.25,0.172080,603.75,12.75,...,0.068343,0.835,0.073,0.661255,33.060001,0.700001,0.228844,14.40,0.50,0.284261
4,87.559998,2.540001,0.253988,0.247827,Buy,1057.25,25.00,0.171875,608.75,13.75,...,0.067522,0.781,0.037,0.651831,33.419998,0.910000,0.228374,14.89,0.33,0.279296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,53.750000,1.850002,0.305261,0.208137,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,55.080002,1.350002,0.302831,0.248413,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,56.799999,1.550003,0.299429,0.268707,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,55.930000,0.990002,0.299686,0.270848,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
for price in Prices_Sb:
    Sugar_H=Sugar_H.merge(price, how="outer", left_index=True, right_index=True)

Sugar_H=Sugar_H.drop("Date", axis=1)    
Sugar_H

,Sugar Close,Sugar Range,Sugar Vol,Price_Change,Buy/Hold,Corn P_Close,Corn P_Range,Corn P_Vol,Wheat P_Close,Wheat P_Range,...,EURUSD P_Vol,IR P_Close,IR P_Range,IR P_Vol,Soybeans P_Close,Soybeans P_Range,Soybeans P_Vol,SoyOil P_Close,SoyOil P_Range,SoyOil P_Vol
0,16.020000,0.180000,0.305977,0.088315,Buy,401.0,8.00,0.234556,608.00,12.50,...,0.067620,0.882,0.025,0.656190,1059.00,17.50,0.172572,33.779999,0.790001,0.230207
1,15.990000,0.340001,0.307256,0.068136,Buy,397.5,4.75,0.234100,607.50,18.50,...,0.068060,0.849,0.027,0.660260,1050.75,11.50,0.172270,33.189999,0.590000,0.229255
2,16.299999,0.480000,0.313401,0.135097,Buy,398.5,5.75,0.236894,598.50,12.00,...,0.068278,0.860,0.049,0.660369,1056.50,15.25,0.172052,33.610001,0.780003,0.228385
3,16.120001,0.370001,0.312884,0.119445,Buy,398.5,11.25,0.236926,603.75,12.75,...,0.068343,0.835,0.073,0.661255,1051.75,11.25,0.172080,33.060001,0.700001,0.228844
4,16.129999,0.230000,0.317853,0.083277,Buy,401.5,14.75,0.237179,608.75,13.75,...,0.067522,0.781,0.037,0.651831,1057.25,25.00,0.171875,33.419998,0.910000,0.228374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,8.350000,0.150000,0.257583,-0.075304,Hold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,8.260000,0.250000,0.258230,-0.099237,Hold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,8.490000,0.139999,0.255229,-0.060841,Hold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,8.390000,0.140000,0.254484,-0.073951,Hold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
for price in Prices_SO:
    SoyOil_H=SoyOil_H.merge(price, how="outer", left_index=True, right_index=True)
    
SoyOil_H=SoyOil_H.drop("Date", axis=1)
SoyOil_H

,SoyOil Close,SoyOil Range,SoyOil Vol,Price_Change,Buy/Hold,Corn P_Close,Corn P_Range,Corn P_Vol,Wheat P_Close,Wheat P_Range,...,EURUSD P_Vol,IR P_Close,IR P_Range,IR P_Vol,Soybeans P_Close,Soybeans P_Range,Soybeans P_Vol,Sugar P_Close,Sugar P_Range,Sugar P_Vol
0,54.259998,1.190002,0.246321,0.606276,Buy,401.0,8.00,0.234556,608.00,12.50,...,0.067620,0.882,0.025,0.656190,1059.00,17.50,0.172572,14.72,0.57,0.291503
1,54.599998,1.399998,0.247860,0.645074,Buy,397.5,4.75,0.234100,607.50,18.50,...,0.068060,0.849,0.027,0.660260,1050.75,11.50,0.172270,14.97,0.81,0.292519
2,55.090000,1.349998,0.248301,0.639095,Buy,398.5,5.75,0.236894,598.50,12.00,...,0.068278,0.860,0.049,0.660369,1056.50,15.25,0.172052,14.36,0.50,0.284243
3,55.090000,0.750000,0.249832,0.666364,Buy,398.5,11.25,0.236926,603.75,12.75,...,0.068343,0.835,0.073,0.661255,1051.75,11.25,0.172080,14.40,0.50,0.284261
4,56.889999,0.000000,0.244642,0.702274,Buy,401.5,14.75,0.237179,608.75,13.75,...,0.067522,0.781,0.037,0.651831,1057.25,25.00,0.171875,14.89,0.33,0.279296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,22.040001,0.380001,0.273128,0.095427,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3953,22.260000,0.230000,0.272905,0.098175,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3954,22.020000,0.230000,0.272758,0.096614,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3955,22.160000,0.120001,0.272803,0.111334,Buy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
Data=[Corn_H, Wheat_H, Cotton_H, Crude_H, USDBRL_H, EURUSD_H, IR_H,Soybeans_H,Sugar_H, SoyOil_H]
client = MongoClient("mongodb://localhost:27017/Group_Project")
db=client.Group_Project


In [51]:
records=json.loads(Soybeans_H.T.to_json()).values()
db.Soybeans.insert(records)

[ObjectId('60538ba80aa32c20e3a4dc7a'),
 ObjectId('60538ba80aa32c20e3a4dc7b'),
 ObjectId('60538ba80aa32c20e3a4dc7c'),
 ObjectId('60538ba80aa32c20e3a4dc7d'),
 ObjectId('60538ba80aa32c20e3a4dc7e'),
 ObjectId('60538ba80aa32c20e3a4dc7f'),
 ObjectId('60538ba80aa32c20e3a4dc80'),
 ObjectId('60538ba80aa32c20e3a4dc81'),
 ObjectId('60538ba80aa32c20e3a4dc82'),
 ObjectId('60538ba80aa32c20e3a4dc83'),
 ObjectId('60538ba80aa32c20e3a4dc84'),
 ObjectId('60538ba80aa32c20e3a4dc85'),
 ObjectId('60538ba80aa32c20e3a4dc86'),
 ObjectId('60538ba80aa32c20e3a4dc87'),
 ObjectId('60538ba80aa32c20e3a4dc88'),
 ObjectId('60538ba80aa32c20e3a4dc89'),
 ObjectId('60538ba80aa32c20e3a4dc8a'),
 ObjectId('60538ba80aa32c20e3a4dc8b'),
 ObjectId('60538ba80aa32c20e3a4dc8c'),
 ObjectId('60538ba80aa32c20e3a4dc8d'),
 ObjectId('60538ba80aa32c20e3a4dc8e'),
 ObjectId('60538ba80aa32c20e3a4dc8f'),
 ObjectId('60538ba80aa32c20e3a4dc90'),
 ObjectId('60538ba80aa32c20e3a4dc91'),
 ObjectId('60538ba80aa32c20e3a4dc92'),
 ObjectId('60538ba80aa32c

In [52]:
records=json.loads(Sugar_H.T.to_json()).values()
db.Sugar.insert(records)

[ObjectId('60538bd50aa32c20e3a4ebef'),
 ObjectId('60538bd50aa32c20e3a4ebf0'),
 ObjectId('60538bd50aa32c20e3a4ebf1'),
 ObjectId('60538bd50aa32c20e3a4ebf2'),
 ObjectId('60538bd50aa32c20e3a4ebf3'),
 ObjectId('60538bd50aa32c20e3a4ebf4'),
 ObjectId('60538bd50aa32c20e3a4ebf5'),
 ObjectId('60538bd50aa32c20e3a4ebf6'),
 ObjectId('60538bd50aa32c20e3a4ebf7'),
 ObjectId('60538bd50aa32c20e3a4ebf8'),
 ObjectId('60538bd50aa32c20e3a4ebf9'),
 ObjectId('60538bd50aa32c20e3a4ebfa'),
 ObjectId('60538bd50aa32c20e3a4ebfb'),
 ObjectId('60538bd50aa32c20e3a4ebfc'),
 ObjectId('60538bd50aa32c20e3a4ebfd'),
 ObjectId('60538bd50aa32c20e3a4ebfe'),
 ObjectId('60538bd50aa32c20e3a4ebff'),
 ObjectId('60538bd50aa32c20e3a4ec00'),
 ObjectId('60538bd50aa32c20e3a4ec01'),
 ObjectId('60538bd50aa32c20e3a4ec02'),
 ObjectId('60538bd50aa32c20e3a4ec03'),
 ObjectId('60538bd50aa32c20e3a4ec04'),
 ObjectId('60538bd50aa32c20e3a4ec05'),
 ObjectId('60538bd50aa32c20e3a4ec06'),
 ObjectId('60538bd50aa32c20e3a4ec07'),
 ObjectId('60538bd50aa32c

In [53]:
records=json.loads(SoyOil_H.T.to_json()).values()
db.SoyOil.insert(records)

[ObjectId('60538bea0aa32c20e3a4fb64'),
 ObjectId('60538bea0aa32c20e3a4fb65'),
 ObjectId('60538bea0aa32c20e3a4fb66'),
 ObjectId('60538bea0aa32c20e3a4fb67'),
 ObjectId('60538bea0aa32c20e3a4fb68'),
 ObjectId('60538bea0aa32c20e3a4fb69'),
 ObjectId('60538bea0aa32c20e3a4fb6a'),
 ObjectId('60538bea0aa32c20e3a4fb6b'),
 ObjectId('60538bea0aa32c20e3a4fb6c'),
 ObjectId('60538bea0aa32c20e3a4fb6d'),
 ObjectId('60538bea0aa32c20e3a4fb6e'),
 ObjectId('60538bea0aa32c20e3a4fb6f'),
 ObjectId('60538bea0aa32c20e3a4fb70'),
 ObjectId('60538bea0aa32c20e3a4fb71'),
 ObjectId('60538bea0aa32c20e3a4fb72'),
 ObjectId('60538bea0aa32c20e3a4fb73'),
 ObjectId('60538bea0aa32c20e3a4fb74'),
 ObjectId('60538bea0aa32c20e3a4fb75'),
 ObjectId('60538bea0aa32c20e3a4fb76'),
 ObjectId('60538bea0aa32c20e3a4fb77'),
 ObjectId('60538bea0aa32c20e3a4fb78'),
 ObjectId('60538bea0aa32c20e3a4fb79'),
 ObjectId('60538bea0aa32c20e3a4fb7a'),
 ObjectId('60538bea0aa32c20e3a4fb7b'),
 ObjectId('60538bea0aa32c20e3a4fb7c'),
 ObjectId('60538bea0aa32c

In [54]:
records=json.loads(Corn_H.T.to_json()).values()
db.Corn.insert(records)

[ObjectId('60538bfd0aa32c20e3a50ad9'),
 ObjectId('60538bfd0aa32c20e3a50ada'),
 ObjectId('60538bfd0aa32c20e3a50adb'),
 ObjectId('60538bfd0aa32c20e3a50adc'),
 ObjectId('60538bfd0aa32c20e3a50add'),
 ObjectId('60538bfd0aa32c20e3a50ade'),
 ObjectId('60538bfd0aa32c20e3a50adf'),
 ObjectId('60538bfd0aa32c20e3a50ae0'),
 ObjectId('60538bfd0aa32c20e3a50ae1'),
 ObjectId('60538bfd0aa32c20e3a50ae2'),
 ObjectId('60538bfd0aa32c20e3a50ae3'),
 ObjectId('60538bfd0aa32c20e3a50ae4'),
 ObjectId('60538bfd0aa32c20e3a50ae5'),
 ObjectId('60538bfd0aa32c20e3a50ae6'),
 ObjectId('60538bfd0aa32c20e3a50ae7'),
 ObjectId('60538bfd0aa32c20e3a50ae8'),
 ObjectId('60538bfd0aa32c20e3a50ae9'),
 ObjectId('60538bfd0aa32c20e3a50aea'),
 ObjectId('60538bfd0aa32c20e3a50aeb'),
 ObjectId('60538bfd0aa32c20e3a50aec'),
 ObjectId('60538bfd0aa32c20e3a50aed'),
 ObjectId('60538bfd0aa32c20e3a50aee'),
 ObjectId('60538bfd0aa32c20e3a50aef'),
 ObjectId('60538bfd0aa32c20e3a50af0'),
 ObjectId('60538bfd0aa32c20e3a50af1'),
 ObjectId('60538bfd0aa32c

In [55]:
records=json.loads(Wheat_H.T.to_json()).values()
db.Wheat.insert(records)

[ObjectId('60538c140aa32c20e3a51a4e'),
 ObjectId('60538c140aa32c20e3a51a4f'),
 ObjectId('60538c140aa32c20e3a51a50'),
 ObjectId('60538c140aa32c20e3a51a51'),
 ObjectId('60538c140aa32c20e3a51a52'),
 ObjectId('60538c140aa32c20e3a51a53'),
 ObjectId('60538c140aa32c20e3a51a54'),
 ObjectId('60538c140aa32c20e3a51a55'),
 ObjectId('60538c140aa32c20e3a51a56'),
 ObjectId('60538c140aa32c20e3a51a57'),
 ObjectId('60538c140aa32c20e3a51a58'),
 ObjectId('60538c140aa32c20e3a51a59'),
 ObjectId('60538c140aa32c20e3a51a5a'),
 ObjectId('60538c140aa32c20e3a51a5b'),
 ObjectId('60538c140aa32c20e3a51a5c'),
 ObjectId('60538c140aa32c20e3a51a5d'),
 ObjectId('60538c140aa32c20e3a51a5e'),
 ObjectId('60538c140aa32c20e3a51a5f'),
 ObjectId('60538c140aa32c20e3a51a60'),
 ObjectId('60538c140aa32c20e3a51a61'),
 ObjectId('60538c140aa32c20e3a51a62'),
 ObjectId('60538c140aa32c20e3a51a63'),
 ObjectId('60538c140aa32c20e3a51a64'),
 ObjectId('60538c140aa32c20e3a51a65'),
 ObjectId('60538c140aa32c20e3a51a66'),
 ObjectId('60538c140aa32c

In [56]:
records=json.loads(Cotton_H.T.to_json()).values()
db.Cotton.insert(records)

[ObjectId('60538c260aa32c20e3a529c3'),
 ObjectId('60538c260aa32c20e3a529c4'),
 ObjectId('60538c260aa32c20e3a529c5'),
 ObjectId('60538c260aa32c20e3a529c6'),
 ObjectId('60538c260aa32c20e3a529c7'),
 ObjectId('60538c260aa32c20e3a529c8'),
 ObjectId('60538c260aa32c20e3a529c9'),
 ObjectId('60538c260aa32c20e3a529ca'),
 ObjectId('60538c260aa32c20e3a529cb'),
 ObjectId('60538c260aa32c20e3a529cc'),
 ObjectId('60538c260aa32c20e3a529cd'),
 ObjectId('60538c260aa32c20e3a529ce'),
 ObjectId('60538c260aa32c20e3a529cf'),
 ObjectId('60538c260aa32c20e3a529d0'),
 ObjectId('60538c260aa32c20e3a529d1'),
 ObjectId('60538c260aa32c20e3a529d2'),
 ObjectId('60538c260aa32c20e3a529d3'),
 ObjectId('60538c260aa32c20e3a529d4'),
 ObjectId('60538c260aa32c20e3a529d5'),
 ObjectId('60538c260aa32c20e3a529d6'),
 ObjectId('60538c260aa32c20e3a529d7'),
 ObjectId('60538c260aa32c20e3a529d8'),
 ObjectId('60538c260aa32c20e3a529d9'),
 ObjectId('60538c260aa32c20e3a529da'),
 ObjectId('60538c260aa32c20e3a529db'),
 ObjectId('60538c260aa32c